In [4]:
import rl_ctrl_sat_aw
from scipy.integrate import ode
from scipy import optimize
import numpy as np
import ipywidgets
import matplotlib.pyplot as plt

In [5]:
%matplotlib widget

## Instantiate system

In [15]:
o = rl_ctrl_sat_aw.rl_ctrl_sat_aw_class()

sld_iref = ipywidgets.FloatSlider(orientation='horizontal',description = u"i_ref:", 
                                value=0.0, min=0.0,max= 10, 
                                step=.5)


## Interaction

In [23]:
o = rl_ctrl_sat_aw.rl_ctrl_sat_aw_class()
o.Dt = 0.001
o.t_end = 5.0
o.decimation = 10
o.update()

o.struct.V_max = 2

o.struct.i_ref = 0
sol = optimize.root(o.run_problem, [1]*(o.N_x+o.N_y), jac=o.dae_jacobian, method='hybr')
o.struct.K_aw = 1.0
x_0 = np.copy(o.struct.x)
y_0 = np.copy(o.struct.y)


fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(7, 7))


line_i = axes[0].plot([],[],label='$\sf i$')
line_v = axes[1].plot([],[],label='$\sf y_{pi}$ (saturated)')

axes[0].set_ylim((-1,12))
axes[1].set_ylim((-0.1,2.1))

for ax in axes:
    ax.grid()
    ax.set_xlim((0,0.5))
    ax.legend()
ax.set_xlabel('Time (s)')


def update(change):
    
    i_ref = sld_iref.value
    
    o.struct.x = np.copy(x_0)
    o.struct.y = np.copy(y_0)
    
    o.struct[0].itol = 1e-6
    o.struct.it_store = 0
    o.struct.t = 0.0
    
    o.struct.i_ref = 0.0
    o.struct.t_end = 0.1
    t = rl_ctrl_sat_aw.daesolver(o.struct)

    o.struct.i_ref = i_ref
    o.struct.t_end = 0.8
    t = rl_ctrl_sat_aw.daesolver(o.struct)
    
    T = o.struct[0]['T'][:o.struct[0].it_store]
    X = o.struct[0].X[:o.struct[0].it_store,:]
    Y = o.struct[0].Y[:o.struct[0].it_store,:]
    Iters = o.struct[0].iters[:o.struct[0].it_store,:]   
    
    line_i[0].set_data(T, X[:,0])
    line_v[0].set_data(T, Y[:,1])
    
sld_iref.observe(update, names='value')
layout = ipywidgets.VBox([sld_iref])
layout

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …